In [3]:
import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [21]:
# source text
data = """ I love recurrent nueral network\n
and convolution nueral network"""

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[3, 4, 5, 1, 2, 6, 7, 1, 2]

In [23]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1

print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 8


In [24]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded_data)):
	sequence = encoded_data[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# split into X and y elements

Total Sequences: 8


In [25]:
sequences
#sequences[:5] # [input, output]

[[3, 4], [4, 5], [5, 1], [1, 2], [2, 6], [6, 7], [7, 1], [1, 2]]

In [26]:
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [27]:
X[:5]

array([3, 4, 5, 1, 2])

In [28]:
y[:5]

array([4, 5, 1, 2, 6])

In [29]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model
y[:5]

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [30]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 10)             80        
                                                                 
 lstm_1 (LSTM)               (None, 50)                12200     
                                                                 
 dense_1 (Dense)             (None, 8)                 408       
                                                                 
Total params: 12688 (49.56 KB)
Trainable params: 12688 (49.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# fit network
model.fit(X, y, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 616ms/step - loss: 2.0795 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0779 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0763 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 2.0748 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0732 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0716 - accuracy: 0.2500
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0700 - accuracy: 0.2500
Epoch 8/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0684 - accuracy: 0.2500
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0668 - accuracy: 0.2500
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 2.0652 - accuracy: 0.2500
Epoch 1

1/1 [==============================] - 0s 2ms/step - loss: 1.7488 - accuracy: 0.8750
Epoch 85/100
1/1 [==============================] - 0s 2ms/step - loss: 1.7395 - accuracy: 0.8750
Epoch 86/100
1/1 [==============================] - 0s 2ms/step - loss: 1.7300 - accuracy: 0.8750
Epoch 87/100
1/1 [==============================] - 0s 3ms/step - loss: 1.7202 - accuracy: 0.8750
Epoch 88/100
1/1 [==============================] - 0s 3ms/step - loss: 1.7103 - accuracy: 0.8750
Epoch 89/100
1/1 [==============================] - 0s 3ms/step - loss: 1.7002 - accuracy: 0.8750
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6899 - accuracy: 0.8750
Epoch 91/100
1/1 [==============================] - 0s 3ms/step - loss: 1.6794 - accuracy: 0.8750
Epoch 92/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6687 - accuracy: 0.8750
Epoch 93/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6578 - accuracy: 0.8750
Epoch 94/100
1/1 [===============

In [33]:
# generate a sequence from the model
def generate_seq(model, tokenizer, enter_text, n_pred):  
	in_text, result = enter_text, enter_text  # 
	# generate a fixed number of words
	for _ in range(n_pred):
        
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
        
		# predict a word in the vocabulary
		yhat = model.predict(encoded)
		pc = np.argmax(yhat, axis=1)
        
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == pc:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [38]:
# evaluate
print(generate_seq(model, tokenizer, 'convolution', 2))

1/1 [==============================] - 0s 8ms/step
convolution nueral network
